### Финальный результат (версия для параграфов)

In [107]:
# импортируем необходимые библиотеки

from docx import Document
from docx.shared import RGBColor
from docx.enum.text import WD_COLOR_INDEX

import numpy as np

from sentence_transformers import SentenceTransformer, util
import statistics as stat

In [158]:
# загрузка данных из файла

def load_data_from_docx(file_name):
    document = Document(file_name)
    words_to_delete=["Характеристики сооружения и основные решения","Характеристики существующего сооружения и основные решения","Мероприятия по обеспечению пожарной безопасности","Антикоррозионная защита","Таблица"]
    rez,found_start=[],False
    for paragraph in document.paragraphs:
        if 'Исходные данные' in paragraph.text:
            found_start = True
        elif found_start:
            rez.append(paragraph.text)
    return [i for i in rez if i.strip() and not any(word in i for word in words_to_delete)]

In [159]:
real_parts=load_data_from_docx("П-07.22-6.1-КР-ТЧ.docx")
#len(real_parts)
#real_parts
data = real_parts

86

In [163]:
# расстояние каждого предложения до других (значения + сортировка)

def generate_seq_of_similarity_inc(data:list)->list:
    results,results_values=[],[]
    embedder = SentenceTransformer('distiluse-base-multilingual-cased')
    for i in range(len(data)):
        SENTENCE_NUM=i
        sentence_embeddings = embedder.encode(data, convert_to_tensor=True)
        given_sentence_embedding = sentence_embeddings[SENTENCE_NUM]
        cosine_similarities_bert = util.pytorch_cos_sim(given_sentence_embedding.reshape(1, -1), sentence_embeddings).numpy().flatten()
        results_values.append(cosine_similarities_bert)
        results.append(cosine_similarities_bert.argsort())
    return results,results_values

In [164]:
results,results_values=generate_seq_of_similarity_inc(data)

In [193]:
# поиск "непохожих" предложений

def get_frequence(results,results_values,isPrint=False):
    sorts=[]
    for j in range(len(results)):
        sorts.append([i for i in results[j] if results_values[j][i]<np.mean(results_values)])
    flat_arr = [val for arr in sorts for val in arr]
    freq_dict = {val: flat_arr.count(val) for val in set(flat_arr)}
    sorted_freq_list=sorted(freq_dict.items(), key=lambda x: x[1], reverse=True)
    global data
    data_stat=[i[1] for i in sorted_freq_list]
    find,mark=[],False
    if isPrint:
        print("\nЧастота встречаемости элементов (статистический анализ):")
        mark=True
    for val, count in sorted_freq_list: 
        if count >= round(stat.mean(data_stat)+1.95*stat.stdev(data_stat)):
            if mark:
                print(f"{val}: {count}")
                print(data[val])
            find.append(data[val])
            
    return sorted_freq_list,find

In [198]:
sorted_freq_list,find=get_frequence(results,results_values,False)
#sorted_freq_list
find

In [197]:
# выгрузка результатов в отдельный файл

def get_review(file_name,find):
    document = Document(file_name)
    # Список предложений, которые нужно выделить
    sentences_to_highlight = find
    # Проходим по всем абзацам в документе
    for paragraph in document.paragraphs:
        # Проверяем, содержит ли абзац какое-либо предложение из списка
        for sentence in sentences_to_highlight:
            if sentence in paragraph.text:
                # Если предложение найдено, выделяем его
                for run in paragraph.runs:
                    if run.text in sentence:
                        #run.font.color.rgb = RGBColor(255, 0, 0)
                        run.font.highlight_color = WD_COLOR_INDEX.YELLOW

    return document.save(f'{file_name}_review.docx')

In [178]:
get_review("П-07.22-6.1-КР-ТЧ.docx",find)

## To do (pdf)